In [ ]:
import pyodbc
import pandas as pd
import numpy as np
conn = pyodbc.connect('Driver={IBM i Access ODBC Driver}; '
       'SYSTEM = 192.168.0.161;'
        'Hostname=M3; '
        'Port=21; '
        'Protocol=TCPIP; '
        'Database=M3; '
        'UID=pud; '
        'PWD = Pud142;'
        ,autocommit=True)

In [ ]:
def clean_and_dict(list_of_data):
    res={}
    for row in list_of_data:
        

        temp=""
        for j in row:
            
            if type(j)==float:
                print(j)
                temp+=str(int(j)).strip()
            else:
                temp+=str(j).strip()
        if temp not in res:
            res[temp]=row
    return res

In [ ]:
sql="SELECT * FROM MATAN.MCWCCO"
# mcwcco=pd.read_sql(sql,conn)
# mcwcco_cols_values=mcwcco.columns.values
corsor=conn.cursor()
mcwcco=cursor.execute(sql).fetchall()
mcwcco_dict=clean_and_dict(mcwcco)


In [240]:
new_data=pd.read_csv("new_data.csv")


In [263]:

_list=[]
for i in range(len(new_data)):
    row=new_data.iloc[i]
    
    temp=[1,int(row[0]),row.cost,int(row.price_type),row.work_center,"","",20221208]
    if row.price_type==1:
        temp.append(row[5] )
    if row.price_type==5:
        temp.append(row[6])
    temp.append(20221114)
    temp.append(130000)
    temp.append(20221114)
    temp.append(1)
    temp.append("it_admin_matan")
    temp=tuple(temp)
    _list.append(temp)



In [262]:
cursor.execute("delete from MATAN.MCWCCO where KECHID='it_admin_matan'")

In [264]:
try:
    conn.autocommit = False
    
    cursor.executemany("insert into MATAN.MCWCCO(KECONO,KEFACI,KECCOM,KEPCTP,KEOBV1,KEOBV2,KEOBV3,KEFRDT,KECDPR,KERGDT,KERGTM,KELMDT,KECHNO,KECHID) values (?, ?,?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", _list)
except pyodbc.DatabaseError as err:
    print(err)
    conn.rollback()
else:
    conn.commit()
finally:
    conn.autocommit = True